In [1]:
# 기본라이브러리 import
#from google.colab import drive
#import os, json, pickle
#import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

#from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.metrics import confusion_matrix
#from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

# 파이토치 라이브러리 import
# import torch
# import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader
# from torch.utils.data import random_split

#Keras Import
import tensorflow as tf
from tensorflow.keras import layers
from datetime import timedelta, datetime

# 데이터 정규화
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

import ipaddress

# 구글 드라이브 mount
#drive.mount('/content/gdrive')

# 데이터 파일 위치
#C:\Users\mariu\내 드라이브\Colab Notebooks\Network
# colab_path = 'C:/Documents/VM/disk/notebook/'
# colab_write_path = "C:/Users/mariu/Downloads/"

In [2]:

df = pd.read_csv("NF-UQ-NIDS-total.csv",index_col=False)


In [3]:
df

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,...,NUM_PKTS_512_TO_1024_BYTES_2,NUM_PKTS_1024_TO_1514_BYTES_2,TCP_WIN_MAX_IN_2,TCP_WIN_MAX_OUT_2,ICMP_TYPE_2,ICMP_IPV4_TYPE_2,DNS_QUERY_ID_2,DNS_QUERY_TYPE_2,DNS_TTL_ANSWER_2,FTP_COMMAND_RET_CODE_2
0,3232261268,65389,3232261127,80,6,7.000,420,3,0,0,...,0,0,0,0,0,0,0,0,0,0.0
1,3232261268,11154,3232261125,80,6,7.000,280,2,40,1,...,0,0,0,0,0,0,0,0,0,0.0
2,3232235807,42062,3232235855,1041,6,0.000,44,1,40,1,...,0,0,1024,0,0,0,0,0,0,0.0
3,3232235810,46849,3232235855,9110,6,0.000,44,1,40,1,...,0,0,1024,0,0,0,0,0,0,0.0
4,3232235806,50360,3232235928,1084,6,0.000,44,1,40,1,...,0,0,4096,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089994,3232235807,32978,3232235960,443,6,0.000,890,7,5060,16,...,0,0,1024,0,0,0,0,0,0,0.0
1089995,3586844392,22736,2887729787,3389,6,0.000,2006,14,2183,11,...,1,0,8192,64000,0,0,0,0,0,0.0
1089996,3232261268,39234,3232261125,80,17,188.000,56,2,0,0,...,0,0,512,0,0,0,0,0,0,0.0
1089997,2887730018,49428,2887712770,53,17,5.126,61,1,77,1,...,0,0,0,0,0,0,0,0,0,0.0


In [4]:
# 확장판에서 사용
#cols = df.columns.drop(['IPV4_SRC_ADDR', 'L4_SRC_PORT', 'IPV4_DST_ADDR', 'L4_DST_PORT', 'PROTOCOL', 'L7_PROTO', 'DNS_QUERY_ID','Label','Attack','Dataset'])  #명목형변수, 레이블 변수 제외
#cols = df.columns.drop(['IPV4_SRC_ADDR_1','IPV4_SRC_ADDR_2','Label','Attack','Dataset', 'Label_1','Attack_1','Dataset_1', 'Label_2','Attack_2','Dataset_2'])  #명목형변수, 레이블 변수 제외
cols = df.columns.drop(['Label','Attack'])  #명목형변수, 레이블 변수 제외#cols = df.columns.drop(['Attack','Dataset'])  #명목형변수, 레이블 변수 제외
X = df[cols]
dummies = pd.get_dummies(df['Attack']) # Classification
outcomes = dummies.columns
num_classes = len(outcomes)
Y = dummies.values

#==================
dummiesLabel = pd.get_dummies(df['Label']) # Classification
y_1_label = df['Label'].values
y_label = dummiesLabel.values

#import pandas as pd
#import numpy as np

def extend_sparse(val):
    if val in ['Analysis', 'Exploits', 'Fuzzers', 'Shellcode', 'Theft', 'Worms', 'mitm']: return 1
    return 0

y_1_attack = pd.DataFrame(df['Attack'])

is_sparse = y_1_attack.applymap(extend_sparse)
y_1_enforce = is_sparse.values
y_enforce = pd.get_dummies(is_sparse['Attack']) # Classification

#=======================

def max_8G(val):
    if (val > 1.0e+9): return 1.0e+9
    return val

#cols =['DST_TO_SRC_SECOND_BYTES', 'SRC_TO_DST_SECOND_BYTES', 'SRC_TO_DST_AVG_THROUGHPUT', 'DST_TO_SRC_AVG_THROUGHPUT']
cols =['DST_TO_SRC_SECOND_BYTES', 'SRC_TO_DST_SECOND_BYTES', 'SRC_TO_DST_AVG_THROUGHPUT', 'DST_TO_SRC_AVG_THROUGHPUT',
'DST_TO_SRC_SECOND_BYTES_1', 'SRC_TO_DST_SECOND_BYTES_1', 'SRC_TO_DST_AVG_THROUGHPUT_1', 'DST_TO_SRC_AVG_THROUGHPUT_1',
'DST_TO_SRC_SECOND_BYTES_2', 'SRC_TO_DST_SECOND_BYTES_2', 'SRC_TO_DST_AVG_THROUGHPUT_2', 'DST_TO_SRC_AVG_THROUGHPUT_2'] 

X[cols] = X[cols].applymap(max_8G)

In [5]:
for col in X.columns:
    print(col , X[col].max())

IPV4_SRC_ADDR 4291722866
L4_SRC_PORT 65535
IPV4_DST_ADDR 4294967295
L4_DST_PORT 65535
PROTOCOL 254
L7_PROTO 248.0
IN_BYTES 34641563
IN_PKTS 123763
OUT_BYTES 129573662
OUT_PKTS 87179
TCP_FLAGS 223
CLIENT_TCP_FLAGS 223
SERVER_TCP_FLAGS 223
FLOW_DURATION_MILLISECONDS 4294967
DURATION_IN 105400
DURATION_OUT 38547
MIN_TTL 255
MAX_TTL 255
LONGEST_FLOW_PKT 7292
SHORTEST_FLOW_PKT 1504
MIN_IP_PKT_LEN 547
MAX_IP_PKT_LEN 7292
SRC_TO_DST_SECOND_BYTES 1000000000.0
DST_TO_SRC_SECOND_BYTES 1000000000.0
RETRANSMITTED_IN_BYTES 6321251
RETRANSMITTED_IN_PKTS 4774
RETRANSMITTED_OUT_BYTES 2183347
RETRANSMITTED_OUT_PKTS 1458
SRC_TO_DST_AVG_THROUGHPUT 1000000000.0
DST_TO_SRC_AVG_THROUGHPUT 1000000000.0
NUM_PKTS_UP_TO_128_BYTES 191858
NUM_PKTS_128_TO_256_BYTES 7230
NUM_PKTS_256_TO_512_BYTES 4921
NUM_PKTS_512_TO_1024_BYTES 34443
NUM_PKTS_1024_TO_1514_BYTES 86096
TCP_WIN_MAX_IN 65535
TCP_WIN_MAX_OUT 65535
ICMP_TYPE 65280
ICMP_IPV4_TYPE 255
DNS_QUERY_ID 65535
DNS_QUERY_TYPE 32769
DNS_TTL_ANSWER 4294915672
FTP_CO

In [6]:
# 데이터 정규화
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV


#scaler_train_minmax = preprocessing.MinMaxScaler()

scaler_train = preprocessing.StandardScaler()
#scaler_train = preprocessing.MinMaxScaler()
scaler_train = scaler_train.fit(X)
X = pd.DataFrame(scaler_train.transform(X),index=np.arange(0,X.shape[0],1), columns = X.columns)
#X = scaler_train.transform(X)


In [7]:
def max_3sigma(val):
    if (val < -3): return -3
    if (val > 3): return 3
    return val

X = X.applymap(max_3sigma)

In [8]:
y1 = tf.argmax(dummies, axis=1)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, shuffle = True, random_state=33)
x_train_label, x_test_label, y_train_label, y_test_label = train_test_split(X, y_label, test_size=0.20, shuffle = True, random_state=33)
x_train_enforce, x_test_enforce, y_train_enforce, y_test_enforce = train_test_split(X, y_enforce, test_size=0.20, shuffle = True, random_state=33)
x_train_1, x_test_1, y_train_1, y_test_1 = train_test_split(X, y1.numpy(), test_size=0.20, shuffle = True, random_state=33)

2023-10-10 02:27:35.423860: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
ㄴㄴ

NameError: name 'ᄂᄂ' is not defined

In [ ]:
#model1
#initializer = "glorot_uniform"
initializer = tf.keras.initializers.GlorotUniform(seed=33)
constraints = None


model_1 = tf.keras.Sequential([
  tf.keras.layers.Dense(64, input_dim=X.shape[1], kernel_initializer=initializer, bias_initializer="zeros", kernel_constraint=constraints, activation='relu',name='dense_1a'),
  tf.keras.layers.Dense(128, input_dim=X.shape[1], kernel_initializer=initializer, bias_initializer="zeros", kernel_constraint=constraints,activation='relu',name='dense_1b'),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(128, input_dim=X.shape[1], kernel_initializer=initializer, bias_initializer="zeros", kernel_constraint=constraints,activation='relu',name='dense_1c'),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(64, input_dim=X.shape[1], kernel_initializer=initializer, bias_initializer="zeros", kernel_constraint=constraints,activation='relu',name='dense_1d'),
  tf.keras.layers.Dense(Y.shape[1],activation='softmax')
  ])
'''
model_1 = tf.keras.Sequential([
  tf.keras.layers.Dense(64, input_dim=X.shape[1], kernel_initializer=initializer, bias_initializer="zeros", kernel_constraint=constraints, activation='relu'),
  tf.keras.layers.Dense(128, input_dim=64, kernel_initializer=initializer, bias_initializer="zeros", kernel_constraint=constraints,activation='relu'),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(128, input_dim=128, kernel_initializer=initializer, bias_initializer="zeros", kernel_constraint=constraints,activation='relu'),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(64, input_dim=128, kernel_initializer=initializer, bias_initializer="zeros", kernel_constraint=constraints,activation='relu'),
  tf.keras.layers.Dense(Y.shape[1],input_dim=64, activation='softmax')
  ])
'''

In [ ]:
#model 1
model_1.compile(loss='categorical_crossentropy',metrics=['accuracy'], optimizer='adam')

log_dir_1 = datetime.now().strftime("%Y%m%d%H%M%S")
print(log_dir_1)
monitor_1 = [
             tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=8, verbose=1, mode='auto',restore_best_weights=True),
#             tf.keras.callbacks.TensorBoard(log_dir=log_dir_1, histogram_freq=1)
#             tf.keras.callbacks.TensorBoard(histogram_freq=1)
            ]
history_1  = model_1.fit(x_train,y_train,validation_data=(x_test,y_test), verbose=2, batch_size=1024, epochs=100)
#history_1  = model_1.fit(x_train,y_train,validation_split=0.2, callbacks=[monitor_1],verbose=2, batch_size=1024, epochs=100)


In [ ]:
#Training  model 2  with different structure
initializer = tf.keras.initializers.GlorotUniform(seed=64)

model_2 = tf.keras.Sequential([
      tf.keras.layers.Dense(64, input_dim=X.shape[1], kernel_initializer=initializer, activation='relu',name='dense_2a'),
      tf.keras.layers.Dense(128, input_dim=64, kernel_initializer=initializer, activation='relu',name='dense_2b'),
      tf.keras.layers.Dropout(0.3),
      tf.keras.layers.Dense(256, input_dim=128, kernel_initializer=initializer, activation='relu',name='dense_2c'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(128, input_dim=256, kernel_initializer=initializer, activation='relu',name='dense_2d'),
      tf.keras.layers.Dropout(0.1),
      tf.keras.layers.Dense(Y.shape[1],activation='softmax')
      ])

In [ ]:
#from tensorflow.keras.callbacks import EarlyStopping

model_2.compile(loss='categorical_crossentropy',metrics=['accuracy'], optimizer='adam')

log_dir_2 = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

#monitor_2 = [
#             tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=8, verbose=1, mode='auto',restore_best_weights=True),
#             tf.keras.callbacks.TensorBoard(log_dir=log_dir_2, histogram_freq=1)
#            ]
#history_2  = model_2.fit(x_train,y_train,validation_data=(x_test,y_test), callbacks=[monitor_2],verbose=2, batch_size=512, epochs=100)
history_2  = model_2.fit(x_train,y_train,validation_data=(x_test,y_test), verbose=2, batch_size=512, epochs=100)

In [ ]:
#3 rd model for stack
model_3 = tf.keras.Sequential([
      tf.keras.layers.Dense(512, input_dim=X.shape[1], kernel_initializer=initializer, activation='relu',name='dense_3a'),
      tf.keras.layers.Dropout(0.1),
      tf.keras.layers.Dense(256, input_dim=X.shape[1], kernel_initializer=initializer, activation='relu',name='dense_3b'),
      tf.keras.layers.Dropout(0.3),
      tf.keras.layers.Dense(128, input_dim=X.shape[1], kernel_initializer=initializer, activation='relu',name='dense_3c'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(128, input_dim=X.shape[1], kernel_initializer=initializer, activation='relu',name='dense_3d'),
      tf.keras.layers.Dense(Y.shape[1],activation='softmax')
      ])

In [ ]:

model_3.compile(loss='categorical_crossentropy',metrics=['accuracy'], optimizer='adam')

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

monitor_3 = [
             tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=8, verbose=2, mode='auto',restore_best_weights=True),
             tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
            ]
history_3  = model_3.fit(x_train,y_train,validation_data=(x_test,y_test), callbacks=[monitor_3],verbose=2, batch_size=512, epochs=100)

In [ ]:
#4 rd model for stack
model_4 = tf.keras.Sequential([
      tf.keras.layers.Dense(128, input_dim=x_train.shape[1], kernel_initializer=initializer, activation='relu',name='dense_4a'),
      tf.keras.layers.Dense(64, input_dim=x_train.shape[1], kernel_initializer=initializer, activation='relu',name='dense_4b'),
      tf.keras.layers.Dropout(0.3),
      tf.keras.layers.Dense(32, input_dim=x_train.shape[1], kernel_initializer=initializer, activation='relu',name='dense_4c'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(128, input_dim=x_train.shape[1], kernel_initializer=initializer, activation='relu',name='dense_4d'),
      tf.keras.layers.Dense(y_train.shape[1],activation='softmax')
      ])

In [ ]:
model_4.compile(loss='categorical_crossentropy',metrics=['accuracy'], optimizer='adam')

log_dir_4 = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

monitor_4 = [
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=100, verbose=1, mode='auto',restore_best_weights=True),
            tf.keras.callbacks.TensorBoard(log_dir=log_dir_4, histogram_freq=1)
        ]
history_4  = model_4.fit(x_train,y_train,validation_data=(x_test,y_test), callbacks=[monitor_4],verbose=2, batch_size=512, epochs=100)

In [ ]:
model_1.save('model1_1.h5')
model_2.save('model1_2.h5')
model_3.save('model1_3.h5')
model_4.save('model1_4.h5')
# model_5.save('model1_5.h5')

In [10]:
# load models from file
# for both logistic and nueral
def load_all_models(n_models):
	all_models = list()

	for i in range(n_models):
		# define filename for this ensemble
		filename = 'model1_' + str(i + 1) + '.h5'
		# load model from file
		model = load_model(filename,custom_objects=None)
		# add to list of members
		all_models.append(model)
		print('>loaded %s' % filename)
	return all_models

In [11]:
# define stacked model from multiple member input models
#neural

def define_stacked_model(members):
	initializer = tf.keras.initializers.GlorotUniform(seed=64)
	constraints = None

  # update all layers in all models to not be trainable
	for i in range(len(members)):
		model = members[i]
		for layer in model.layers:
			# make not trainable
			layer.trainable = False
			# rename to avoid 'unique layer name' issue
			layer._name = 'ensemble_' + str(i + 1) + '_' + layer.name


	# define multi-headed input
	ensemble_visible = [model.input for model in members]
	# concatenate merge output from each model
	ensemble_outputs = [model.output for model in members]
	merge = concatenate(ensemble_outputs)
	hidden1 = Dense(128, kernel_initializer=initializer, activation='relu',name="dense_ea")(merge)
	hidden2 = Dense(64, kernel_initializer=initializer, activation='relu',name="dense_eb")(hidden1)
	output = Dense(Y.shape[1], kernel_initializer=initializer, activation='softmax',name="dense_ec")(hidden1)
	model = Model(inputs=ensemble_visible, outputs=output)
	# plot graph of ensemble
	plot_model(model, show_shapes=True, to_file='model_graph.png')
	# compile
	model.summary()
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [20]:
# fit a stacked model
#neural
import datetime


#colab_path = "gdrive/My Drive/Colab Notebooks/Network/models/"
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

monitor_6= [
             tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
            ]

def fit_stacked_model(model, inputX, inputy):
	# prepare input data
	X = [inputX for _ in range(len(model.input))]
	# encode output data
	inputy_enc = to_categorical(inputy)
	# fit model
	history_6=model.fit(X, inputy_enc, epochs=60, verbose=2,callbacks=[monitor_6])

In [21]:
# monitor_6= [
#              tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
#             ]

In [22]:
# # fit a stacked model
# #neural
# def fit_stacked_model(model, inputX, inputy):
# 	# prepare input data
# 	X = [inputX for _ in range(len(model.input))]
# 	# encode output data
# 	inputy_enc = to_categorical(inputy)
# 	# fit model
# 	history_6=model.fit(X, inputy_enc, epochs=32, verbose=2,callbacks=[monitor_6])

In [23]:
# make a prediction with a stacked model
#neural
def predict_stacked_model(model, inputX):
	# prepare input data
	X = [inputX for _ in range(len(model.input))]
	# make prediction
	return model.predict(X, verbose=2)

In [24]:
# load all models
#neural
from keras.models import load_model
n_members = 4
members = load_all_models(n_members)
print('Loaded %d models' % len(members))

>loaded model1_1.h5
>loaded model1_2.h5
>loaded model1_3.h5
>loaded model1_4.h5
Loaded 4 models


In [25]:
# define ensemble model
import matplotlib.pyplot as plt
#from keras.utils import plot_model
#neural
#from keras.layers.merge import concatenate
from keras.layers import concatenate
from keras.layers import Dense
from keras.models import Model
from tensorflow.keras.utils import plot_model

stacked_model = define_stacked_model(members)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 dense_2a_input (InputLayer)    [(None, 127)]        0           []                               
                                                                                                  
 dense_3a_input (InputLayer)    [(None, 127)]        0           []                               
                                                                                                  
 dense_1a_input (InputLayer)    [(None, 127)]        0           []                               
                                                                                                  
 ensemble_2_dense_2a (Dense) 

In [26]:
from keras.utils import to_categorical
inputy_enc = to_categorical(y_test_1)
print(y_test_1.shape)
print(inputy_enc.shape)
y_test_1

(218000,)
(218000, 21)


array([ 5,  6,  2, ...,  2, 19,  2])

In [27]:
from keras.utils import to_categorical
# fit stacked model on test dataset
#neural
fit_stacked_model(stacked_model, x_test_1, y_test_1)

Epoch 1/60
6813/6813 - 10s - loss: 0.0954 - accuracy: 0.9820 - 10s/epoch - 1ms/step
Epoch 2/60
6813/6813 - 9s - loss: 0.0696 - accuracy: 0.9837 - 9s/epoch - 1ms/step
Epoch 3/60
6813/6813 - 8s - loss: 0.0670 - accuracy: 0.9835 - 8s/epoch - 1ms/step
Epoch 4/60
6813/6813 - 8s - loss: 0.0657 - accuracy: 0.9837 - 8s/epoch - 1ms/step
Epoch 5/60
6813/6813 - 8s - loss: 0.0650 - accuracy: 0.9837 - 8s/epoch - 1ms/step
Epoch 6/60
6813/6813 - 8s - loss: 0.0642 - accuracy: 0.9838 - 8s/epoch - 1ms/step
Epoch 7/60
6813/6813 - 8s - loss: 0.0642 - accuracy: 0.9838 - 8s/epoch - 1ms/step
Epoch 8/60
6813/6813 - 8s - loss: 0.0640 - accuracy: 0.9837 - 8s/epoch - 1ms/step
Epoch 9/60
6813/6813 - 8s - loss: 0.0636 - accuracy: 0.9839 - 8s/epoch - 1ms/step
Epoch 10/60
6813/6813 - 8s - loss: 0.0632 - accuracy: 0.9837 - 8s/epoch - 1ms/step
Epoch 11/60
6813/6813 - 8s - loss: 0.0629 - accuracy: 0.9837 - 8s/epoch - 1ms/step
Epoch 12/60
6813/6813 - 8s - loss: 0.0633 - accuracy: 0.9839 - 8s/epoch - 1ms/step
Epoch 13/60

In [28]:
from sklearn.metrics import accuracy_score
import numpy as np
# make predictions and evaluate
#neural
yhat = predict_stacked_model(stacked_model, x_test_1)
yhat_val = tf.argmax(yhat, axis=1)
acc = accuracy_score(y_test_1, yhat_val)
print('Stacked Test Accuracy: %.4f' % acc)

6813/6813 - 6s - 6s/epoch - 895us/step
Stacked Test Accuracy: 0.9846


In [29]:
from sklearn.metrics import classification_report
print(classification_report(y_test_1, yhat_val))
classification_report(y_test_1, yhat_val)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       1.00      0.94      0.97        51
           2       0.99      0.99      0.99     72091
           3       1.00      1.00      1.00       433
           4       1.00      0.97      0.98       365
           5       0.99      0.99      0.99     62503
           6       0.98      0.99      0.99     51385
           7       0.51      0.91      0.66        78
           8       0.52      0.25      0.34        67
           9       0.98      0.73      0.83        62
          10       0.97      0.21      0.34       349
          11       0.98      0.95      0.96      7418
          12       0.80      0.80      0.80         5
          13       0.78      0.78      0.78         9
          15       0.91      0.75      0.82      1989
          16       1.00      0.47      0.64        17
          17       0.94      0.91      0.93      3272
          18       1.00    

'              precision    recall  f1-score   support\n\n           0       0.00      0.00      0.00         3\n           1       1.00      0.94      0.97        51\n           2       0.99      0.99      0.99     72091\n           3       1.00      1.00      1.00       433\n           4       1.00      0.97      0.98       365\n           5       0.99      0.99      0.99     62503\n           6       0.98      0.99      0.99     51385\n           7       0.51      0.91      0.66        78\n           8       0.52      0.25      0.34        67\n           9       0.98      0.73      0.83        62\n          10       0.97      0.21      0.34       349\n          11       0.98      0.95      0.96      7418\n          12       0.80      0.80      0.80         5\n          13       0.78      0.78      0.78         9\n          15       0.91      0.75      0.82      1989\n          16       1.00      0.47      0.64        17\n          17       0.94      0.91      0.93      3272\n       